# Using Jupyter to write FastHTML

To wrote FastHTML applications in Jupyter notebooks requires a slightly different process than normal Python applications. 

:::{.callout-tip title='Download this notebook and try it yourself'}
The source code for this page is a [Jupyter notebook](https://github.com/AnswerDotAI/fasthtml/blob/main/nbs/tutorials/jupyter_and_fasthtml.ipynb). That makes it easy to directly experiment with it. However, as this is working code that means we have to comment out a few things in order for the documentation to build.
:::

## Getting started

The first step is to import necessary libraries. As using FastHTML inside a Jupyter notebook is a special case, it remains a special import. 

In [ ]:
from fasthtml.common import *
from fasthtml.jupyter import *

Let's use `jupy_app` to instantiate `app` and `rt` objects. `jupy_app` is a thin wrapper around the `fast_app` function.

In [ ]:
app, rt = jupy_app()

Define a route to test the application.

In [ ]:
@rt
def index():
    return Titled('Hello, Jupyter',
           P('Welcome to the FastHTML + Jupyter example')
    )

Set the `port` in a variable so it can be more easily shared between the `server` and `HTMX` symbols described below.

In [ ]:
port = 8000

Create a `server` object using `JupyUvi`, which also starts Uvicorn. The `server` runs in a separate thread from Jupyter, so it can use normal HTTP client functions in a notebook.

In [ ]:
# Run locally, uncomment this line to create and start the server
# server = JupyUvi(app, port=port)

The `HTMX` callable displays the server's HTMX application in an iframe which can be displayed by Jupyter notebook. Pass in the same `port` variable used in the `JupyUvi` callable above.

In [ ]:
# Run locally, uncomment this line to display the HTMX application in Jupyter
HTMX(port=port)

## Refreshing the HTMX() view

If we change the above view, we don't need to run `server = JupyUvi(app, port=port)` again. All we need to do is refresh the cell that contains `HTMX(port=port)`. This also applies to other tasks like importing more Python libraries, adding functions, general debugging, and more. To try it out:

1. Change the `Titled()` FT Component from "Hello, Jupyter" to say "Hello, FastHTML"
2. Refresh the Jupyter cell containing `HTMX(port=port) 

## Full screen view


You can view your app outside of Jupyter by going to `localhost:PORT`, where `PORT` is what you set the `port` variable to. This can be useful for debugging. Unless you change the `port` value in this tutorial, just click on this link: [localhost:8000/](http://localhost:8000/).

Understand that this is FastHTML running in Jupyter mode, so things will look different. For example, unlike `fast_app`, `jupy_app` doesn't include Picoweb CSS as a default.

## Graceful shutdowns

Use the `server.stop()` function displayed below. If you restart Jupyter without calling this line the thread may not be released and the `HTMX` callable above may throw errors. If that happens, a quick temporary fix is to change the `port` number above to something else.

Cleaner solutions to the dangling thread are to kill the dangling thread (dependant on each operating system) or restart the computer.

In [ ]:
# Run locally, uncomment this line to stop the server
# server.stop()